In [1]:
import tensorflow as tf

In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [4]:
input_file = '../data/baidu_95.csv'

In [7]:
df = pd.read_csv(input_file, header=None).rename(columns={0:"labels", 1:"content"})

In [8]:
df.head()

,labels,content
0,高中 生物 分子与细胞 组成细胞的化学元素 组成细胞的化合物,菠菜从土壤中吸收的氮元素可以用来合成（）A.淀粉和纤维素B.葡萄糖和DNAC.核酸和蛋白质D...
1,高中 生物 稳态与环境 神经调节和体液调节的比较,下列有关生物体内信息传递的叙述，正确的是（）A下丘脑分泌的促甲状腺激素释放激素，可作用于甲状...
2,高中 生物 生物技术实践 生物工程技术,从自然菌样筛选较理想生产菌种的一般步骤是：采集菌样→富集培养→纯种分离→性能测定．1.不同微...
3,高中 生物 生物技术实践 生物技术在其他方面的应用 器官移植 复等位基因 胚胎移植 基因工程...,目前，精子载体法逐渐成为具有诱惑力的制备转基因动物方法之一，该方法以精子作为外源基因的载体，...
4,高中 地理 宇宙中的地球 地球运动的地理意义,某人想乘普通飞机在一年中连续过两次生日，你认为应穿越（）A赤道B两级C本初子午线D国际日期变更线


In [9]:
df.count()

labels     22576
content    22576
dtype: int64

In [10]:
df.describe()

,labels,content
count,22576,22576
unique,673,18419
top,高中 地理 宇宙中的地球 地球运动的地理意义,下列四幅图中，能正确表示地球自转方向的是（）ABCD
freq,1533,12


In [13]:
df["subject"] = df['labels'].apply(lambda x:x.split()[1])

In [14]:
df.head()

,labels,content,subject
0,高中 生物 分子与细胞 组成细胞的化学元素 组成细胞的化合物,菠菜从土壤中吸收的氮元素可以用来合成（）A.淀粉和纤维素B.葡萄糖和DNAC.核酸和蛋白质D...,生物
1,高中 生物 稳态与环境 神经调节和体液调节的比较,下列有关生物体内信息传递的叙述，正确的是（）A下丘脑分泌的促甲状腺激素释放激素，可作用于甲状...,生物
2,高中 生物 生物技术实践 生物工程技术,从自然菌样筛选较理想生产菌种的一般步骤是：采集菌样→富集培养→纯种分离→性能测定．1.不同微...,生物
3,高中 生物 生物技术实践 生物技术在其他方面的应用 器官移植 复等位基因 胚胎移植 基因工程...,目前，精子载体法逐渐成为具有诱惑力的制备转基因动物方法之一，该方法以精子作为外源基因的载体，...,生物
4,高中 地理 宇宙中的地球 地球运动的地理意义,某人想乘普通飞机在一年中连续过两次生日，你认为应穿越（）A赤道B两级C本初子午线D国际日期变更线,地理


In [15]:
def clean_sentence(line):
    line = re.sub(
            "[a-zA-Z0-9]|[\s+\-\|\!\/\[\]\{\}_,.$%^*(+\"\')]+|[:：+——()?【】《》“”！，。？、~@#￥%……&*（）]+|题目", '',line)
    words = jieba.cut(line, cut_all=False)
    
    return words

In [16]:
stopwords_path = "../data/stopwords/哈工大停用词表.txt"

In [22]:
def load_stop_words(stopwords_path):
    with open(stopwords_path, 'r') as f:
        stop_words = [word.strip() for word in f.readlines()]
        stop_words_dict = dict(zip(stop_words, list(range(len(stop_words)))))
    
    return stop_words_dict

In [35]:
from collections import Counter
import re
import jieba
import json

In [45]:
def sentence_proc(sentence):
    # 清除无用词
    cw_words = list(clean_sentence(sentence))
    print("after clear sentence:", len(list(cw_words)))
    
    # 去除停用词
    sw_words = [word for word in cw_words if word not in load_stop_words(stopwords_path)]
    print("after clear stop_words:", len(sw_words))
    
    return " ".join(sw_words)

In [64]:
def generate_vocab(contents, labels):
    # 拉平词语
    all_words = [word for content in contents for word in content]
    print("all words:", len(all_words))
    # 去除低频词
    word_counts = Counter(all_words)
    sorted_words = sorted(word_counts.items(), key=lambda x:x[1], reverse=True)
    fw_words = [word[0] for word in sorted_words if word[1] > 5]
    print("after clear low frequence:", len(fw_words))
    
    # 生成词表
    word2idx = dict(zip(fw_words, list(range(len(fw_words)))))
    
    # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
    with open("../data/word2idx.json", "w", encoding="utf-8") as f:
        json.dump(word2idx, f)
        
    return word2idx

In [47]:
df['content'] = df['content'].apply(sentence_proc)

after clear sentence: 22
after clear stop_words: 15
after clear sentence: 56
after clear stop_words: 41
after clear sentence: 259
after clear stop_words: 184
after clear sentence: 147
after clear stop_words: 103
after clear sentence: 20
after clear stop_words: 16
after clear sentence: 64
after clear stop_words: 46
after clear sentence: 71
after clear stop_words: 46
after clear sentence: 45
after clear stop_words: 33
after clear sentence: 72
after clear stop_words: 58
after clear sentence: 10
after clear stop_words: 8
after clear sentence: 36
after clear stop_words: 26
after clear sentence: 61
after clear stop_words: 51
after clear sentence: 18
after clear stop_words: 13
after clear sentence: 195
after clear stop_words: 137
after clear sentence: 18
after clear stop_words: 16
after clear sentence: 49
after clear stop_words: 37
after clear sentence: 40
after clear stop_words: 21
after clear sentence: 64
after clear stop_words: 40
after clear sentence: 20
after clear stop_words: 17
after c

after clear stop_words: 53
after clear sentence: 14
after clear stop_words: 12
after clear sentence: 81
after clear stop_words: 70
after clear sentence: 176
after clear stop_words: 134
after clear sentence: 12
after clear stop_words: 9
after clear sentence: 41
after clear stop_words: 36
after clear sentence: 50
after clear stop_words: 36
after clear sentence: 11
after clear stop_words: 7
after clear sentence: 18
after clear stop_words: 15
after clear sentence: 95
after clear stop_words: 72
after clear sentence: 40
after clear stop_words: 29
after clear sentence: 43
after clear stop_words: 36
after clear sentence: 39
after clear stop_words: 30
after clear sentence: 27
after clear stop_words: 25
after clear sentence: 161
after clear stop_words: 110
after clear sentence: 171
after clear stop_words: 138
after clear sentence: 29
after clear stop_words: 26
after clear sentence: 51
after clear stop_words: 39
after clear sentence: 92
after clear stop_words: 73
after clear sentence: 16
after cl

after clear stop_words: 27
after clear sentence: 54
after clear stop_words: 40
after clear sentence: 54
after clear stop_words: 37
after clear sentence: 47
after clear stop_words: 39
after clear sentence: 75
after clear stop_words: 45
after clear sentence: 32
after clear stop_words: 17
after clear sentence: 26
after clear stop_words: 21
after clear sentence: 261
after clear stop_words: 183
after clear sentence: 17
after clear stop_words: 15
after clear sentence: 34
after clear stop_words: 20
after clear sentence: 31
after clear stop_words: 21
after clear sentence: 29
after clear stop_words: 17
after clear sentence: 76
after clear stop_words: 56
after clear sentence: 105
after clear stop_words: 79
after clear sentence: 54
after clear stop_words: 39
after clear sentence: 92
after clear stop_words: 69
after clear sentence: 47
after clear stop_words: 24
after clear sentence: 19
after clear stop_words: 13
after clear sentence: 56
after clear stop_words: 46
after clear sentence: 73
after cle

after clear stop_words: 61
after clear sentence: 71
after clear stop_words: 56
after clear sentence: 77
after clear stop_words: 50
after clear sentence: 25
after clear stop_words: 15
after clear sentence: 32
after clear stop_words: 24
after clear sentence: 97
after clear stop_words: 84
after clear sentence: 64
after clear stop_words: 45
after clear sentence: 33
after clear stop_words: 22
after clear sentence: 43
after clear stop_words: 36
after clear sentence: 37
after clear stop_words: 29
after clear sentence: 40
after clear stop_words: 25
after clear sentence: 85
after clear stop_words: 49
after clear sentence: 49
after clear stop_words: 35
after clear sentence: 49
after clear stop_words: 43
after clear sentence: 46
after clear stop_words: 24
after clear sentence: 35
after clear stop_words: 28
after clear sentence: 76
after clear stop_words: 48
after clear sentence: 77
after clear stop_words: 53
after clear sentence: 64
after clear stop_words: 47
after clear sentence: 17
after clear 

after clear stop_words: 59
after clear sentence: 20
after clear stop_words: 14
after clear sentence: 383
after clear stop_words: 283
after clear sentence: 42
after clear stop_words: 37
after clear sentence: 201
after clear stop_words: 156
after clear sentence: 87
after clear stop_words: 70
after clear sentence: 62
after clear stop_words: 45
after clear sentence: 54
after clear stop_words: 48
after clear sentence: 26
after clear stop_words: 24
after clear sentence: 97
after clear stop_words: 76
after clear sentence: 72
after clear stop_words: 59
after clear sentence: 47
after clear stop_words: 28
after clear sentence: 218
after clear stop_words: 148
after clear sentence: 41
after clear stop_words: 32
after clear sentence: 49
after clear stop_words: 34
after clear sentence: 12
after clear stop_words: 10
after clear sentence: 26
after clear stop_words: 16
after clear sentence: 37
after clear stop_words: 31
after clear sentence: 39
after clear stop_words: 31
after clear sentence: 70
after 

...

after clear stop_words: 42
after clear sentence: 144
after clear stop_words: 103
after clear sentence: 22
after clear stop_words: 17
after clear sentence: 36
after clear stop_words: 28
after clear sentence: 52
after clear stop_words: 41
after clear sentence: 16
after clear stop_words: 15
after clear sentence: 44
after clear stop_words: 39
after clear sentence: 31
after clear stop_words: 21
after clear sentence: 31
after clear stop_words: 24
after clear sentence: 52
after clear stop_words: 37
after clear sentence: 10
after clear stop_words: 8
after clear sentence: 19
after clear stop_words: 16
after clear sentence: 41
after clear stop_words: 33
after clear sentence: 36
after clear stop_words: 28
after clear sentence: 35
after clear stop_words: 21
after clear sentence: 53
after clear stop_words: 41
after clear sentence: 53
after clear stop_words: 43
after clear sentence: 18
after clear stop_words: 14
after clear sentence: 32
after clear stop_words: 22
after clear sentence: 58
after clear

after clear stop_words: 156
after clear sentence: 23
after clear stop_words: 17
after clear sentence: 94
after clear stop_words: 69
after clear sentence: 88
after clear stop_words: 52
after clear sentence: 48
after clear stop_words: 34
after clear sentence: 24
after clear stop_words: 21
after clear sentence: 61
after clear stop_words: 49
after clear sentence: 25
after clear stop_words: 21
after clear sentence: 60
after clear stop_words: 50
after clear sentence: 86
after clear stop_words: 61
after clear sentence: 41
after clear stop_words: 33
after clear sentence: 252
after clear stop_words: 172
after clear sentence: 30
after clear stop_words: 17
after clear sentence: 17
after clear stop_words: 13
after clear sentence: 51
after clear stop_words: 42
after clear sentence: 80
after clear stop_words: 58
after clear sentence: 78
after clear stop_words: 60
after clear sentence: 86
after clear stop_words: 68
after clear sentence: 27
after clear stop_words: 19
after clear sentence: 118
after cl

after clear stop_words: 21
after clear sentence: 44
after clear stop_words: 31
after clear sentence: 194
after clear stop_words: 130
after clear sentence: 161
after clear stop_words: 133
after clear sentence: 47
after clear stop_words: 28
after clear sentence: 84
after clear stop_words: 59
after clear sentence: 49
after clear stop_words: 38
after clear sentence: 31
after clear stop_words: 26
after clear sentence: 206
after clear stop_words: 148
after clear sentence: 49
after clear stop_words: 28
after clear sentence: 98
after clear stop_words: 69
after clear sentence: 99
after clear stop_words: 82
after clear sentence: 58
after clear stop_words: 49
after clear sentence: 66
after clear stop_words: 46
after clear sentence: 41
after clear stop_words: 29
after clear sentence: 17
after clear stop_words: 14
after clear sentence: 69
after clear stop_words: 50
after clear sentence: 34
after clear stop_words: 29
after clear sentence: 65
after clear stop_words: 45
after clear sentence: 13
after 

after clear stop_words: 66
after clear sentence: 62
after clear stop_words: 54
after clear sentence: 42
after clear stop_words: 24
after clear sentence: 32
after clear stop_words: 24
after clear sentence: 9
after clear stop_words: 3
after clear sentence: 13
after clear stop_words: 10
after clear sentence: 61
after clear stop_words: 37
after clear sentence: 54
after clear stop_words: 42
after clear sentence: 36
after clear stop_words: 25
after clear sentence: 89
after clear stop_words: 72
after clear sentence: 75
after clear stop_words: 50
after clear sentence: 43
after clear stop_words: 39
after clear sentence: 59
after clear stop_words: 38
after clear sentence: 47
after clear stop_words: 35
after clear sentence: 55
after clear stop_words: 38
after clear sentence: 46
after clear stop_words: 34
after clear sentence: 54
after clear stop_words: 43
after clear sentence: 79
after clear stop_words: 59
after clear sentence: 26
after clear stop_words: 20
after clear sentence: 145
after clear s

after clear stop_words: 135
after clear sentence: 153
after clear stop_words: 104
after clear sentence: 45
after clear stop_words: 35
after clear sentence: 16
after clear stop_words: 13
after clear sentence: 49
after clear stop_words: 41
after clear sentence: 58
after clear stop_words: 43
after clear sentence: 65
after clear stop_words: 53
after clear sentence: 47
after clear stop_words: 25
after clear sentence: 24
after clear stop_words: 18
after clear sentence: 58
after clear stop_words: 49
after clear sentence: 140
after clear stop_words: 117
after clear sentence: 35
after clear stop_words: 29
after clear sentence: 53
after clear stop_words: 44
after clear sentence: 14
after clear stop_words: 11
after clear sentence: 44
after clear stop_words: 24
after clear sentence: 62
after clear stop_words: 47
after clear sentence: 108
after clear stop_words: 68
after clear sentence: 17
after clear stop_words: 13
after clear sentence: 140
after clear stop_words: 108
after clear sentence: 15
afte

In [48]:
df.head()

,labels,content,subject
0,高中 生物 分子与细胞 组成细胞的化学元素 组成细胞的化合物,菠菜 土壤 中 吸收 氮 元素 用来 合成 淀粉 纤维素 葡萄糖 核酸 蛋白质 麦芽糖 脂肪酸,生物
1,高中 生物 稳态与环境 神经调节和体液调节的比较,下列 生物体 内 信息 传递 叙述 正确 下丘脑 分泌 促 甲状腺 激素 释放 激素 作用 ...,生物
2,高中 生物 生物技术实践 生物工程技术,自然 菌样 筛选 理想 生产 菌种 步骤 采集 菌样 富集 培养 纯种 分离 性能 测定 不...,生物
3,高中 生物 生物技术实践 生物技术在其他方面的应用 器官移植 复等位基因 胚胎移植 基因工程...,目前 精子 载体 法 逐渐 成为 具有 诱惑力 制备 转基因 动物 方法 方法 精子 外源 ...,生物
4,高中 地理 宇宙中的地球 地球运动的地理意义,某人 想 普通 飞机 一年 中 连续 两次 生日 认为 应 穿越 赤道 两级 本初子午线 国...,地理


In [66]:
contents = df["content"].tolist()
len(contents)

22576

In [67]:
contents = [content.split() for content in contents]

In [68]:
contents[:1]

[['菠菜',
  '土壤',
  '中',
  '吸收',
  '氮',
  '元素',
  '用来',
  '合成',
  '淀粉',
  '纤维素',
  '葡萄糖',
  '核酸',
  '蛋白质',
  '麦芽糖',
  '脂肪酸']]

In [69]:
labels = df["subject"].tolist()

In [70]:
word2idx = generate_vocab(contents, labels)

all words: 1019403
after clear low frequence: 12602


In [71]:
word2idx

{'中': 0,
 '细胞': 1,
 '下列': 2,
 '基因': 3,
 '染色体': 4,
 '正确': 5,
 '过程': 6,
 '不': 7,
 '上': 8,
 '叙述': 9,
 '时': 10,
 '进行': 11,
 '实验': 12,
 '都': 13,
 '产生': 14,
 '后': 15,
 '主要': 16,
 '发展': 17,
 '表示': 18,
 '材料': 19,
 '发生': 20,
 '﹣': 21,
 '中国': 22,
 '植物': 23,
 '基因型': 24,
 '问题': 25,
 '内': 26,
 '不同': 27,
 '经济': 28,
 '年': 29,
 '作用': 30,
 '可能': 31,
 '生物': 32,
 '杂交': 33,
 '形成': 34,
 '地球': 35,
 '我国': 36,
 '错误': 37,
 '如图': 38,
 '图': 39,
 '回答': 40,
 '变化': 41,
 '出现': 42,
 '酶': 43,
 '技术': 44,
 '人': 45,
 '使': 46,
 '植株': 47,
 '具有': 48,
 '分别': 49,
 '遗传': 50,
 '物质': 51,
 '相同': 52,
 '蛋白质': 53,
 '属于': 54,
 '影响': 55,
 '原因': 56,
 '控制': 57,
 '个体': 58,
 '分裂': 59,
 '太阳': 60,
 '正常': 61,
 '分离': 62,
 '社会': 63,
 '方法': 64,
 '胚胎': 65,
 '调节': 66,
 '结构': 67,
 '文化': 68,
 '需要': 69,
 '°': 70,
 '现象': 71,
 '说法': 72,
 '培养': 73,
 '动物': 74,
 '目的': 75,
 '性状': 76,
 '环境': 77,
 '分析': 78,
 '会': 79,
 '利用': 80,
 '下': 81,
 '含有': 82,
 '一定': 83,
 '激素': 84,
 '获得': 85,
 '分子': 86,
 '位于': 87,
 '果蝇': 88,
 '抗体': 89,
 '比例': 90,
 '显性': 91,
 '免疫': 92,
